In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import random 
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader,random_split
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.manifold import TSNE
from vae import VariationalAutoencoder
from modules import train_epoch, test_epoch, plot_ae_outputs


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')

In [ ]:
# dataset = torchvision.datasets.ImageFolder("../media/")
dataset = torchvision.datasets.MNIST(root="", train=True, download=True, transform=None)
train_transform = transforms.Compose([
    transforms.ToTensor(),
])

dataset.transform = train_transform

m=len(dataset)
trainset, valset = random_split(dataset, [int(m-0.2*m), int(0.2*m)])


train_loader = DataLoader(trainset)
val_loader = DataLoader(valset)

In [ ]:
in_channels = len(dataset[0][0])
d = 128
lr = 1e-3 
model = VariationalAutoencoder(in_channels, d, device)
optim = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
model.to(device)

In [ ]:
num_epochs = 100

for epoch in range(num_epochs):
   train_loss = train_epoch(model,device,train_loader,optim)
   val_loss = test_epoch(model,device,val_loader)
   print('\n EPOCH {}/{} \t train loss {:.3f} \t val loss {:.3f}'.format(epoch + 1, num_epochs,train_loss,val_loss))
   plot_ae_outputs(model.encoder,model.decoder,valset,device,n=2)